In [19]:
import arcpy
from arcpy import env
arcpy.env.workspace = r'C:\Users\ltmsbi\Documents\ArcGIS\Projects\Final_Deployment\Final_Deployment.gdb'
arcpy.env.OverwriteOutput = True
species = input("Enter species shortname (e.g., phrag): ")
threshold = "precision"
presence_feature = "SVI_Project_presences_"+species+"_"+threshold

Enter species shortname (e.g., phrag): phrag


In [3]:
# Add & calculate new date field
def add_formatted_date_field(feature):
    arcpy.management.AddField(feature, "date_formatted", "DATE", field_alias="date_formatted")
    arcpy.management.CalculateField(feature, "date_formatted", "!date!")

add_formatted_date_field(presence_feature)
add_formatted_date_field("pred_finalDeployment_all")

In [3]:
# Spatial join n years of presences to distinct layers

# Define field mappings to keep only the target layer's fields
def create_SJ_field_mappings(targetLayer, joinLayer): # Return field mappings for spatial joins when called 
    
    # List starting fields for spatial joins that'll be updated with each successive join
    keepFields = list()
    omitFields = ["OBJECTID", "Shape", "Shape_Area", "Shape_Length"]
    for field in arcpy.ListFields(targetLayer):
        if field.name not in omitFields:
            keepFields.append(field.name)
    fieldMappings = arcpy.FieldMappings() # field mapping variable; this will store all field mappings
    # Create list of field names to keep in the output file
    targetTable = []
    for i in arcpy.ListFields(targetLayer):
        if i.name in (keepFields):
            targetTable.append(i.name)
    # List of input feature classes for the spatial join
    f = [targetLayer, joinLayer]
    # loop through main table
    for k in targetTable: 
        #print("Field: ",k)
        fieldMap = arcpy.FieldMap() # create an empty field map variable
        fieldMap.addInputField(targetLayer,k) # insert the target layer as the first input into the field map
        for feature in f: # loop through feature classes
            for field in arcpy.ListFields(feature): # loop through field of each feature class
                if k in field.name: # check if any field matches with our target field then append it as an input field
                    fieldMap.addInputField(feature,field.name) 
        fieldMappings.addFieldMap(fieldMap) # add the current field map to the main field map variable
    return(fieldMappings)
# Get range of presence dates for which to run spatial joins
def year_range(presences):
    import datetime
    import time
    listDates = []
    cursor = arcpy.da.SearchCursor(presences, field_names="date_formatted")
    for row in cursor:
        listDates.append(row)
    # Get max and min datetime objects, isolate year as integer
    max_date = max(listDates)[0].year
    min_date = min(listDates)[0].year
    year_range = [min_date,max_date+1]
    return year_range

year_range = year_range(presence_feature)
panel_features = list()
hex_grid = "SVI_Proj_hex_grid_"+species

for year in range(year_range[0],year_range[1]):
    time_clause = "date_formatted >= timestamp '"+str(year)+"-01-01 00:00:00' And date_formatted <= timestamp '"+str(year)+"-12-31 23:59:59'"
    # Join year's model pano count to hex grid
    pano_points = arcpy.management.SelectLayerByAttribute("pred_finalDeployment_all", "NEW_SELECTION", time_clause)
    hex_grid_SJ = "tmp_hex_"+str(year)
    fm = create_SJ_field_mappings(hex_grid, pano_points) # May need to delete some fields here
    arcpy.analysis.SpatialJoin(hex_grid, pano_points, hex_grid_SJ, field_mapping=fm)
    del fm
    arcpy.management.DeleteField(hex_grid_SJ, "TARGET_FID")
    arcpy.management.AlterField(hex_grid_SJ, "JOIN_COUNT", "model_points", "model_points")
    # Isolate year's presences as join layer
    join_layer = arcpy.management.SelectLayerByAttribute(presence_feature, "NEW_SELECTION", time_clause)
    # Spatial join model data and presences
    fm = create_SJ_field_mappings(hex_grid_SJ, join_layer)
    tmp_join = "SJ_"+species+"_"+threshold+"_"+str(year)
    arcpy.analysis.SpatialJoin(hex_grid_SJ, join_layer, tmp_join, field_mapping=fm)
    # Add & assign timestamp
    d_field = "date"
    arcpy.management.AddField(tmp_join, d_field, "DATE", field_alias=d_field)
    arcpy.management.CalculateField(tmp_join, d_field, "'"+str(year)+"-01-01 12:00:00 AM'")
    # Add & calculate prevalence
    prev_field = species+"_prev"
    arcpy.management.AddField(tmp_join, prev_field, "FLOAT", field_alias=prev_field)
    arcpy.management.CalculateField(tmp_join, prev_field, "!JOIN_COUNT!/!model_points!")
    # Add field for filling later
    prev_field_filled = species+"_prev_fill"
    arcpy.management.AddField(tmp_join, prev_field_filled, "FLOAT", field_alias=prev_field_filled)
    # Delete JOIN_COUNT, and yearly pano count fields
    # arcpy.management.DeleteField(in_table=tmp_join,drop_field=['JOIN_COUNT','model_points'])
    # Append layer to panel features for merging later
    panel_features.append(tmp_join)
    arcpy.management.SelectLayerByAttribute(presence_feature, "CLEAR_SELECTION")
    del time_clause, join_layer, fm, tmp_join, d_field, prev_field

# Merge into one feature for filling and conversion to netCDF
panel_data_unfilled = "SVI_Proj_panel_data_"+threshold+"_"+species+"_unfilled"
arcpy.management.Merge(panel_features, panel_data_unfilled)

<Result 'C:\\Users\\ltmsbi\\Documents\\ArcGIS\\Projects\\Final_Deployment\\Final_Deployment.gdb\\SVI_Proj_panel_data_precision_phrag'>

In [24]:
panel_data_unfilled = "SVI_Proj_panel_data_"+threshold+"_"+species+"_unfilled"
# Fill zero prevalence values with last nonzero value

# Convert panel features to pandas sedf
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
unfilled_sedf = pd.DataFrame.spatial.from_featureclass(panel_data_unfilled)
# Sort by two columns to make sure the time step 
# ids are in order based on location id
unfilled_sedf.sort_values(by=["GRID_ID", "date"],
                          ascending = [True, True],
                          inplace=True)
# Define and apply function to fill zeroes with last nonzero value
def fill_values(df):
    new_parr = []
    fill = 0
    # Return the sum of pano join count for the current Location ID
    count_sum = df['model_points'].sum()
    for val in df[species+"_prev"]:
        if count_sum == 0:
            fill = np.nan
        elif count_sum > 0:
            # Set the fill value to be the last positive value encountered
            if val > 0:
                fill = val
        else:
            pass
        new_parr.append(fill)
    df[species+"_prev"] = new_parr
    return df
filled_sedf = unfilled_sedf.groupby('GRID_ID').apply(fill_values)
#filled_sedf.iloc[0:50,:]
print("Prevalence values filled")
# Convert back to feature class
panel_data_filled = arcpy.env.workspace+r'\SVI_Proj_panel_data_'+threshold+"_"+species
filled_sedf.spatial.to_featureclass(panel_data_filled)
print("Panel feature class created")
# Convert to netCDF that's ready for emerging hotspot analysis
arcpy.stpm.CreateSpaceTimeCubeDefinedLocations(in_features=panel_data_filled, 
                                               output_cube='SVI_Proj_panel_cube_'+threshold+"_"+species, 
                                               location_id='target_fid', 
                                               temporal_aggregation='NO_TEMPORAL_AGGREGATION', 
                                               time_field='date',
                                               time_step_interval='1 Years',
                                               variables=[[species+'_prev','DROP_LOCATIONS']]
                                              )
print("Space-time cube complete and located in project folder")

Prevalence values filled
Panel feature class created
Space-time cube complete


In [17]:
# Proof-of-concept for filling behavior
import pandas as pd
import numpy as np
d = {'COUNT': [ 0, 1, 0, 5, 0 , 2 , 0, 0 ,  0,  0,  0,  0], 
     'date': [1, 2, 3, 4, 1, 2, 3, 4, 1, 2, 3, 4], 
     'Location ID': [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3]
    }
df = pd.DataFrame(data=d)
def fill_values(small_df):
    new_parr = []
    fill = 0
    # Return the sum of values for the current Location ID
    count_sum = small_df['COUNT'].sum()
    # Fill values according to logic
    for val in small_df['COUNT']:
        if count_sum == 0:
            fill = np.nan
        elif count_sum > 0:
            # Set the fill value to be the last positive value encountered
            if val > 0:
                fill = val
        else:
            pass
        new_parr.append(fill)
    small_df["COUNT"] = new_parr
    return small_df
# Apply the fill_values function to each Location ID group.
# Applied in this way, the function fills NaN values in locations 
# with only zero values for COUNT (i.e., locations with no panoramas
# assessed during any timestep). It fills empty COUNT timesteps with 
# the last positive value for locations with any positive values, or 
# zero if there had not yet been any positive values (i.e., panoramas 
# assessed)
result = df.groupby('Location ID').apply(fill_values)
result

,COUNT,date,Location ID
0,0.0,1,1
1,1.0,2,1
2,1.0,3,1
3,5.0,4,1
4,0.0,1,2
5,2.0,2,2
6,2.0,3,2
7,2.0,4,2
8,NaN,1,3
9,NaN,2,3
